# Getting, Cleaning, Exporting Data
Source: [kaggle](https://www.kaggle.com/datasets/smadler92/nfl-pfr)
The Dataset comes in multiple Tables that will need to be merged. We have Data from 1920 up until 2021 (but only two games from 2021, so it should be cut to 2020). These tables are organized:
* By Team Folder
    * 4 Files - Roster, Stats by Year, Weekly Odds, Weekly Scores
    * Weekly Injuries Folder
        * Each Year (2009-2021)
* Weather File

Want to be working with the weeklly scores from each team, adding a new column for the team name, and adding them to a DataFrame which will contain all teams data.

In [1]:
# Libraries
import pandas as pd # working with DataFrames
import numpy as np # linear algebra
import os #navigating the folders and files
import glob
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

Need to Creating a Function that will navigate through our team folders (Ex: ATL)

In [2]:
# Function for pulling csv files and adding to single dataframe
def join_frames(save=False, verbose=True):
    
    # Extract teams and filenames from data folder
    csv_files = [] # Empty list to store the file names
    teams = [] # empty list to hold all team abbreviations
    for filename in Path('data/NFLML').glob('**/* WeeklyScores All Years.csv'):
        csv_files.append(filename)
        team_abr = str(filename).split(' ')[0].split('\\')[3] # Only selects the team abbreviation from the file name
        teams.append(team_abr)
    
    data = [] #empty list to store the dataframes
    
    #loop through index up to 32
    for i in range(len(teams)):
        if verbose:
            print(f"Getting Weekly Scores for {teams[i]}") # verbose
        team_df = pd.read_csv(csv_files[i], index_col=0)
        team_df['Team'] = teams[i] # new column for each DataFrames associated team
        data.append(team_df)
    
    df = pd.concat(data, axis=0).reset_index() # concatenate all the separate teams dfs
    if save:
        df.to_csv('data/nfl_scores_merged.csv', index=False)
    return df, teams

In [3]:
# Check if the merged csv file already exists:
merged_file = 'data/nfl_scores_merged.csv'
if os.path.exists(merged_file):
    print("File already exists. Loading in the CSV")
    df = pd.read_csv(merged_file)
else:
    print("File does not exist. Will create DataFrame and save CSV")
    df, teams = join_frames(save=True)

File already exists. Loading in the CSV


In [4]:
print(f"This combined scores DataFrame contains {df.shape[0]} rows and {df.shape[1]} columns.") # get the size of the initial DataFrame

This combined scores DataFrame contains 33890 rows and 26 columns.


In [5]:
# Checking basic information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33890 entries, 0 to 33889
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Week                     33208 non-null  object 
 1   Day                      32200 non-null  object 
 2   Date                     32882 non-null  object 
 3   Time                     25066 non-null  object 
 4   Outcome                  31720 non-null  object 
 5   OT                       33890 non-null  bool   
 6   Rec                      31720 non-null  object 
 7   Home                     33890 non-null  object 
 8   Opp                      33208 non-null  object 
 9   Score_Tm                 31720 non-null  float64
 10  Score_Opp                31720 non-null  float64
 11  Offense_1stD             30637 non-null  float64
 12  Offense_TotYd            30647 non-null  float64
 13  Offense_PassY            30619 non-null  float64
 14  Offense_RushY         

In [6]:
df.tail()

Week  Day         Date       Time Outcome     OT  Rec   Home  \
33885   14  Sun  December 12  1:00PM ET     NaN  False  NaN   True   
33886   15  Sun  December 19  1:00PM ET     NaN  False  NaN  False   
33887   16  Sun  December 26  8:20PM ET     NaN  False  NaN  False   
33888   17  Sun    January 2  1:00PM ET     NaN  False  NaN   True   
33889   18  Sun    January 9  1:00PM ET     NaN  False  NaN  False   

                       Opp  Score_Tm  ...  Defense_1stD  Defense_TotYd  \
33885       Dallas Cowboys       NaN  ...           NaN            NaN   
33886  Philadelphia Eagles       NaN  ...           NaN            NaN   
33887       Dallas Cowboys       NaN  ...           NaN            NaN   
33888  Philadelphia Eagles       NaN  ...           NaN            NaN   
33889      New York Giants       NaN  ...           NaN            NaN   

       Defense_PassY  Defense_RushY  Defense_TO  Year  \
33885            NaN            NaN         NaN  2021   
33886            NaN            NaN         NaN  2021   
33887            NaN            NaN         NaN  2021   
33888            NaN            NaN         NaN  2021   
33889            NaN            NaN         NaN  2021   

       Expected Points_Offense  Expected Points_Defense  \
33885                      NaN                      NaN   
33886                      NaN                      NaN   
33887                      NaN                      NaN   
33888                      NaN                      NaN   
33889                      NaN                      NaN   

       Expected Points_Sp. Tms  Team  
33885                      NaN   WAS  
33886                      NaN   WAS  
33887                      NaN   WAS  
33888                      NaN   WAS  
33889                      NaN   WAS  

[5 rows x 26 columns]

In [7]:
df.loc[df['Year']==2021]

Week  Day          Date       Time Outcome     OT  Rec   Home  \
918      1  Sun  September 12  1:00PM ET       L  False  0-1   True   
919      2  Sun  September 19  4:05PM ET       L  False  0-2  False   
920      3  Sun  September 26  1:00PM ET     NaN  False  NaN  False   
921      4  Sun     October 3  1:00PM ET     NaN  False  NaN   True   
922      5  Sun    October 10  9:30AM ET     NaN  False  NaN   True   
...    ...  ...           ...        ...     ...    ...  ...    ...   
33885   14  Sun   December 12  1:00PM ET     NaN  False  NaN   True   
33886   15  Sun   December 19  1:00PM ET     NaN  False  NaN  False   
33887   16  Sun   December 26  8:20PM ET     NaN  False  NaN  False   
33888   17  Sun     January 2  1:00PM ET     NaN  False  NaN   True   
33889   18  Sun     January 9  1:00PM ET     NaN  False  NaN  False   

                            Opp  Score_Tm  ...  Defense_1stD  Defense_TotYd  \
918         Philadelphia Eagles       6.0  ...          24.0          434.0   
919        Tampa Bay Buccaneers      25.0  ...          21.0          341.0   
920             New York Giants       NaN  ...           NaN            NaN   
921    Washington Football Team       NaN  ...           NaN            NaN   
922               New York Jets       NaN  ...           NaN            NaN   
...                         ...       ...  ...           ...            ...   
33885            Dallas Cowboys       NaN  ...           NaN            NaN   
33886       Philadelphia Eagles       NaN  ...           NaN            NaN   
33887            Dallas Cowboys       NaN  ...           NaN            NaN   
33888       Philadelphia Eagles       NaN  ...           NaN            NaN   
33889           New York Giants       NaN  ...           NaN            NaN   

       Defense_PassY  Defense_RushY  Defense_TO  Year  \
918            261.0          173.0         NaN  2021   
919            259.0           82.0         1.0  2021   
920              NaN            NaN         NaN  2021   
921              NaN            NaN         NaN  2021   
922              NaN            NaN         NaN  2021   
...              ...            ...         ...   ...   
33885            NaN            NaN         NaN  2021   
33886            NaN            NaN         NaN  2021   
33887            NaN            NaN         NaN  2021   
33888            NaN            NaN         NaN  2021   
33889            NaN            NaN         NaN  2021   

       Expected Points_Offense  Expected Points_Defense  \
918                     -12.48                   -13.14   
919                     -15.96                    -8.63   
920                        NaN                      NaN   
921                        NaN                      NaN   
922                        NaN                      NaN   
...                        ...                      ...   
33885                      NaN                      NaN   
33886                      NaN                      NaN   
33887                      NaN                      NaN   
33888                      NaN                      NaN   
33889                      NaN                      NaN   

       Expected Points_Sp. Tms  Team  
918                       2.36   ATL  
919                       0.72   ATL  
920                        NaN   ATL  
921                        NaN   ATL  
922                        NaN   ATL  
...                        ...   ...  
33885                      NaN   WAS  
33886                      NaN   WAS  
33887                      NaN   WAS  
33888                      NaN   WAS  
33889                      NaN   WAS  

[576 rows x 26 columns]

We can see above that the tail end of our DataFrame conatins many NaN values. This is due to the dataset containing up until the year 2021, yet only having data on the first two games of that season. Therefore, I will be dropping all rows from the 2021 season to keep it simple, without losing too much data. 

In [8]:
df = df.loc[df['Year']!=2021]
df.shape

(33314, 26)

## Missing Values

In [9]:
df.isna().sum()

Week                         682
Day                         1658
Date                         976
Time                        8792
Outcome                     1658
OT                             0
Rec                         1658
Home                           0
Opp                          682
Score_Tm                    1658
Score_Opp                   1658
Offense_1stD                2741
Offense_TotYd               2731
Offense_PassY               2759
Offense_RushY               2736
Offense_TO                  7346
Defense_1stD                2741
Defense_TotYd               2731
Defense_PassY               2760
Defense_RushY               2737
Defense_TO                  7349
Year                           0
Expected Points_Offense    19132
Expected Points_Defense    19132
Expected Points_Sp. Tms    19132
Team                           0
dtype: int64

Things to notice:
- We are missing values from Week (this could be because at the beginning of playoffs each year, each team has a separator row in their data).

In [10]:
# Looking at missing week data
df[df['Week'].isna()].Date.unique()

array(['Playoffs'], dtype=object)

We can see here that all of our missing 'Week' values come from these separator rows. Therefore they can be safely dropped

In [11]:
# Drop rows where week is NaN
df.dropna(subset=['Week'], inplace=True)
df.shape # check that we are 682 less rows

(32632, 26)

Next, I will look at the `Day` that are missing values

In [12]:
df[df['Day'].isna()].sample(5)

Week  Day Date Time Outcome     OT  Rec  Home       Opp  Score_Tm  ...  \
28393    2  NaN  NaN  NaN     NaN  False  NaN  True  Bye Week       NaN  ...   
17033    7  NaN  NaN  NaN     NaN  False  NaN  True  Bye Week       NaN  ...   
27900    7  NaN  NaN  NaN     NaN  False  NaN  True  Bye Week       NaN  ...   
14308    9  NaN  NaN  NaN     NaN  False  NaN  True  Bye Week       NaN  ...   
24219    4  NaN  NaN  NaN     NaN  False  NaN  True  Bye Week       NaN  ...   

       Defense_1stD  Defense_TotYd  Defense_PassY  Defense_RushY  Defense_TO  \
28393           NaN            NaN            NaN            NaN         NaN   
17033           NaN            NaN            NaN            NaN         NaN   
27900           NaN            NaN            NaN            NaN         NaN   
14308           NaN            NaN            NaN            NaN         NaN   
24219           NaN            NaN            NaN            NaN         NaN   

       Year  Expected Points_Offense  Expected Points_Defense  \
28393  2008                      NaN                      NaN   
17033  1993                      NaN                      NaN   
27900  2006                      NaN                      NaN   
14308  2010                      NaN                      NaN   
24219  2009                      NaN                      NaN   

       Expected Points_Sp. Tms  Team  
28393                      NaN   RAV  
17033                      NaN   MIN  
27900                      NaN   RAM  
14308                      NaN   JAX  
24219                      NaN   PHI  

[5 rows x 26 columns]

It appears that this is all due to when a team has a 'bye-week'. Let's first confirm this, and if it turns out to be the case, then these rows missing `Day` can be safely dropped.

In [13]:
df[df['Day'].isna()]['Opp'].unique() # Check that the only value for 'Opp' is 'Bye Week'

# Can safely drop the rows containg Day=NaN
df.dropna(subset=['Day'], inplace=True)

I have a feeling that these 'Bye Week' were causing a lot of missing values in our dataset. Let's take another overview of the misisng values:

In [14]:
# Check missing values
df.isna().sum()

Week                           0
Day                            0
Date                           0
Time                        7134
Outcome                        0
OT                             0
Rec                            0
Home                           0
Opp                            0
Score_Tm                       0
Score_Opp                      0
Offense_1stD                1083
Offense_TotYd               1073
Offense_PassY               1101
Offense_RushY               1078
Offense_TO                  5688
Defense_1stD                1083
Defense_TotYd               1073
Defense_PassY               1102
Defense_RushY               1079
Defense_TO                  5691
Year                           0
Expected Points_Offense    17474
Expected Points_Defense    17474
Expected Points_Sp. Tms    17474
Team                           0
dtype: int64

My assumption was correct, and it appears many of our missing values are gone! But definitely some important ones remain, such as time and Offensive and Defensive Stats. We also have a LOT of missing values for `Expected` stats, but because these are going to be irrelelvant to our analysis, it is safe to simply drop these columns.

In [15]:
# Drop the 3 'Expected' Columns
cols_to_drop = ['Expected Points_Offense', 'Expected Points_Defense', 'Expected Points_Sp. Tms']
df.drop(columns=cols_to_drop, inplace=True)
df.head()

Week  Day          Date Time Outcome     OT  Rec   Home  \
0    1  Sun  September 11  NaN       L  False  0-1   True   
1    2  Sun  September 18  NaN       L  False  0-2  False   
2    3  Sun  September 25  NaN       L  False  0-3  False   
3    4  Sun     October 2  NaN       L  False  0-4   True   
4    5  Sun     October 9  NaN       L  False  0-5  False   

                   Opp  Score_Tm  ...  Offense_PassY  Offense_RushY  \
0     Los Angeles Rams      14.0  ...          116.0          121.0   
1  Philadelphia Eagles      10.0  ...          202.0           96.0   
2        Detroit Lions      10.0  ...          104.0          155.0   
3       Dallas Cowboys      14.0  ...          170.0          106.0   
4  Washington Redskins      20.0  ...          146.0          122.0   

   Offense_TO  Defense_1stD  Defense_TotYd  Defense_PassY  Defense_RushY  \
0         2.0          23.0          421.0          275.0          146.0   
1         3.0          20.0          340.0          135.0          205.0   
2         3.0          18.0          344.0          208.0          136.0   
3         5.0          22.0          363.0          220.0          143.0   
4         2.0          26.0          432.0          286.0          146.0   

   Defense_TO  Year  Team  
0         2.0  1966   ATL  
1         2.0  1966   ATL  
2         3.0  1966   ATL  
3         2.0  1966   ATL  
4         NaN  1966   ATL  

[5 rows x 23 columns]

In [16]:
# Check if remainder of missing values is related to the year data was gotten
sorted(list(df.loc[df['Time'].isna()]['Year'].unique()))

[1920,
 1921,
 1922,
 1923,
 1924,
 1925,
 1926,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969]

In [17]:
# Check if remainder of missing values is related to the year data was gotten
df.loc[df['Offense_1stD'].isna()]['Year'].unique()

array([1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930,
       1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1966, 1942],
      dtype=int64)

In [18]:
# Check if remainder of missing values is related to the year data was gotten
df.loc[df['Defense_1stD'].isna()]['Year'].unique()

array([1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930,
       1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1966, 1942],
      dtype=int64)

#### Drop Rows before 1970
Due to the missing values that take place before 1970, along with data before that point being somewhat irrelevant to todays game, I have opted to dropping the years below 1970.

In [19]:
df_1970 = df.loc[df['Year']>1969].copy() #only get games from 1970 and above

In [20]:
df_1970.isna().sum()

Week                0
Day                 0
Date                0
Time                0
Outcome             0
OT                  0
Rec                 0
Home                0
Opp                 0
Score_Tm            0
Score_Opp           0
Offense_1stD        0
Offense_TotYd       0
Offense_PassY       8
Offense_RushY       1
Offense_TO       4201
Defense_1stD        0
Defense_TotYd       0
Defense_PassY       8
Defense_RushY       1
Defense_TO       4201
Year                0
Team                0
dtype: int64

In [21]:
print(f"Our DataFrame now contains {df_1970.shape[0]} rows and {df_1970.shape[1]} columns.")

Our DataFrame now contains 24522 rows and 23 columns.


## Remvoing Duplicate Games
An interest artifact of my data is that all the games technically have duplicates. This is due to the fact that my data was originally organized by each team, so you have a game where, for example, in 1966 Atlanta Plays LA. We have the same game from LA's point of view, where LA plays Atlanta. To circumvent this issue, I am going to create a unique identifier code for each game.

The UniqueID will be of the format year,week,home,away ( An example for the ATL vs LAR: 196601ATLLAR).

To do this, I will need to maniuplate my DataFrame such that I have the Home and Away team for each game. This can be done by using the `Home` Column, in conjunction with the `Team` and `Opp` Columns. Furthermore, the `Opp` column needs to be converted from the Full Team Name, to the Team's Abbreviation.

Steps:
1. Get list of team abbreviations (already done in my concatenating files at beginning of notebook). 
1. Get list of unique team names from the `Opp` column.
1. create dictionary of full name to abbr
1. Map the `Opp` column using this dictionary.
1. Determine create column in DataFrame for home and away team (using the `Home` column as boolean condition).
1. create new column for `GameID` that joins columns: `Year`,`Week`,`HomeTeam`,`AwayTeam`
1. Drop duplicates based on this `GameID`

#### Convert Full Team Names into their Abbreviations

In [22]:
df_1970.head() # quick peak at current working dataframe

Week  Day          Date       Time Outcome     OT  Rec   Home  \
56    1  Sun  September 20  1:30PM ET       W  False  1-0  False   
57    2  Sun  September 27  2:00PM ET       L  False  1-1  False   
58    3  Sun     October 4  1:00PM ET       W  False  2-1   True   
59    4  Sun    October 11  2:00PM ET       L  False  2-2  False   
60    5  Sun    October 18  4:00PM ET       L  False  2-3  False   

                    Opp  Score_Tm  ...  Offense_PassY  Offense_RushY  \
56   New Orleans Saints      14.0  ...          116.0          149.0   
57    Green Bay Packers      24.0  ...          272.0          112.0   
58  San Francisco 49ers      21.0  ...          204.0          152.0   
59       Dallas Cowboys       0.0  ...           44.0           84.0   
60       Denver Broncos      10.0  ...          140.0           76.0   

    Offense_TO  Defense_1stD  Defense_TotYd  Defense_PassY  Defense_RushY  \
56         2.0           9.0          235.0          150.0           85.0   
57         5.0          15.0          344.0          258.0           86.0   
58         2.0          18.0          326.0          163.0          163.0   
59         3.0          13.0          257.0           39.0          218.0   
60         5.0          18.0          223.0          132.0           91.0   

    Defense_TO  Year  Team  
56         2.0  1970   ATL  
57         3.0  1970   ATL  
58         2.0  1970   ATL  
59         3.0  1970   ATL  
60         1.0  1970   ATL  

[5 rows x 23 columns]

In [23]:
# This cell is to search for paticular team names, and see when they played (for helping with changing team names)
df_1970.loc[df_1970['Opp'] == 'Las Vegas Raiders'].Year.unique() # replace team name with name of interest

array([2020], dtype=int64)

In [24]:
# Check the unique opponent names
opp_names = list(df_1970.Opp.unique())
sorted(opp_names)

['Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Colts',
 'Baltimore Ravens',
 'Boston Patriots',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Oilers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Raiders',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Oakland Raiders',
 'Philadelphia Eagles',
 'Phoenix Cardinals',
 'Pittsburgh Steelers',
 'San Diego Chargers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'St. Louis Cardinals',
 'St. Louis Rams',
 'Tampa Bay Buccaneers',
 'Tennessee Oilers',
 'Tennessee Titans',
 'Washington Football Team',
 'Washington Redskins']

In [25]:
# How many Unique Team names (since 1970)
df_1970.Opp.nunique()

43

This is 11 more teams than we have in our current abbreviations `teams` variable (32). This is due to some teams changing there names over the years. Though this is inconvenient, it is certainly manageable. 

These are the Changes:
* Baltimore Colts -> Indianapolis Colts (1984)
* Boston Patriots -> New England Patriots (1971)
* Houston Oilers -> Tennessee Oilers (1997)
* Tennessee Oilers -> Tennessee Titans (1999)
* San Diego Chargers -> Los Angeles Chargers (2017)
* Oakland Raiders -> Los Angeles Raiders (1982)
* Los Anageles Raiders -> Oakland Raiders (1995)
* Oakland Raiders -> Las Vegas Raiders (2020)
* St. Louis Cardinals -> Phoenix Cardinals (1988)
* Phoenix Cardinals -> Arizona Cardinals (1994)
* Los Angeles Rams -> St. Louis Rams (1995)
* St. Louis Rams -> Los Angeles Rams (2016)

I was planning on writing a script to help with mapping Full Team names to their Abbreviation. But since I have them all to see, and have to deal with changing team names, I will just enter this mapping dictionary 'manually' to save some time. I may *come back to this* at a later date to revise and clean up this process.

In [26]:
# Create a dictionary mapping team names to their abbreviation
team_mapping = {'Arizona Cardinals' : 'CRD',
                'Atlanta Falcons' : 'ATL',
                'Baltimore Colts' : 'CLT',
                'Baltimore Ravens' : 'RAV',
                'Boston Patriots' : 'NWE',
                'Buffalo Bills' : 'BUF',
                'Carolina Panthers' : 'CAR',
                'Chicago Bears' : 'CHI',
                'Cincinnati Bengals' : 'CIN',
                'Cleveland Browns' : 'CLE',
                'Dallas Cowboys' : 'DAL',
                'Denver Broncos' : 'DEN',
                'Detroit Lions' : 'DET',
                'Green Bay Packers' : 'GNB',
                'Houston Oilers' : 'OTI',
                'Houston Texans' : 'HTX',
                'Indianapolis Colts' : 'CLT',
                'Jacksonville Jaguars' : 'JAX',
                'Kansas City Chiefs' : 'KAN',
                'Las Vegas Raiders' : 'RAI',
                'Los Angeles Chargers' : 'SDG',
                'Los Angeles Raiders' : 'RAI',
                'Los Angeles Rams' : 'RAM',
                'Miami Dolphins' : 'MIA',
                'Minnesota Vikings' : 'MIN',
                'New England Patriots' : 'NWE',
                'New Orleans Saints' : 'NOR',
                'New York Giants' : 'NYG',
                'New York Jets' : 'NYJ',
                'Oakland Raiders' : 'RAI',
                'Philadelphia Eagles' : 'PHI',
                'Phoenix Cardinals' : 'CRD',
                'Pittsburgh Steelers' : 'PIT',
                'San Diego Chargers' : 'SDG',
                'San Francisco 49ers' : 'SFO',
                'Seattle Seahawks' : 'SEA',
                'St. Louis Cardinals' : 'CRD',
                'St. Louis Rams' : 'RAM',
                'Tampa Bay Buccaneers': 'TAM',
                'Tennessee Oilers' : 'OTI',
                'Tennessee Titans' : 'OTI',
                'Washington Football Team' : 'WAS',
                'Washington Redskins' : 'WAS'
               }

In [27]:
print(f"So we can see that we have {len(team_mapping.keys())} dictionary keys, and {len(set(team_mapping.values()))} unique dictionary values to use for mapping.")
print("This is equivalent to the values that we found before, so we are good for mapping!")

So we can see that we have 43 dictionary keys, and 32 unique dictionary values to use for mapping.
This is equivalent to the values that we found before, so we are good for mapping!


In [28]:
# Create new column and use the dictionary to map Opp name to abbreviated version
df_1970['OppAbbr'] = df_1970.Opp.map(team_mapping)

In [29]:
df_1970.sample(10)

Week  Day          Date       Time Outcome     OT  Rec   Home  \
7175    13  Sun    December 6  1:00PM ET       W  False  8-4  False   
21401    6  Mon    October 14  9:00PM ET       W  False  5-1   True   
33675   14  Sun   December 13  4:05PM ET       W  False  4-9  False   
17794   10  Sun    November 6  1:00PM ET       W  False  6-4   True   
6293     1  Sun  September 20  4:00PM ET       W  False  1-0  False   
14343   10  Thu    November 8  8:30PM ET       L  False  1-8   True   
11873   10  Sun   November 12  1:03PM ET       L  False  2-7   True   
685      5  Sun     October 5  1:02PM ET       W  False  3-2  False   
14163    5  Sun     October 6  4:14PM ET       W  False  3-1   True   
1941     9  Sun    November 8  1:00PM ET       W  False  7-2   True   

                       Opp  Score_Tm  ...  Offense_RushY  Offense_TO  \
7175        Houston Texans      26.0  ...          109.0         NaN   
21401       Miami Dolphins      23.0  ...          245.0         NaN   
33675      Oakland Raiders      34.0  ...          100.0         1.0   
17794      Atlanta Falcons      27.0  ...          238.0         1.0   
6293    San Diego Chargers      16.0  ...          101.0         3.0   
14343   Indianapolis Colts      10.0  ...           37.0         3.0   
11873  San Francisco 49ers      13.0  ...           71.0         4.0   
685      Green Bay Packers      27.0  ...          176.0         1.0   
14163  Philadelphia Eagles      28.0  ...           70.0         NaN   
1941      Seattle Seahawks      44.0  ...           34.0         NaN   

       Defense_1stD  Defense_TotYd  Defense_PassY  Defense_RushY  Defense_TO  \
7175           20.0          398.0          308.0           90.0         2.0   
21401          13.0          200.0          126.0           74.0         1.0   
33675          15.0          227.0          162.0           65.0         1.0   
17794          14.0          213.0          120.0           93.0         3.0   
6293           15.0          288.0          205.0           83.0         1.0   
14343          24.0          359.0          221.0          138.0         2.0   
11873          17.0          315.0          117.0          198.0         1.0   
685            21.0          408.0          304.0          104.0         1.0   
14163          28.0          406.0          207.0          199.0         NaN   
1941           22.0          419.0          362.0           57.0         4.0   

       Year  Team  OppAbbr  
7175   2020   CLT      HTX  
21401  1985   NYJ      MIA  
33675  2009   WAS      RAI  
17794  1983   NOR      ATL  
6293   1970   CLT      SDG  
14343  2012   JAX      CLT  
11873  2006   DET      SFO  
685    2008   ATL      GNB  
14163  2002   JAX      PHI  
1941   2020   BUF      SEA  

[10 rows x 24 columns]

#### Separating Team and Opp into Home and Away Team

In [30]:
# Check that Home column is a boolean of True or False
df_1970.Home.unique()

array(['False', 'True', 'N'], dtype=object)

Notice that there is a 'N' value. Let's investigate this further:

In [31]:
# Check for the value of 'N' in the Home column
df_1970.loc[df_1970['Home']=='N'].sort_values(by='Year')

Week  Day        Date       Time Outcome     OT     Rec Home  \
8817   SuperBowl  Sun  January 17  1:50PM ET       L  False    12-5    N   
6310   SuperBowl  Sun  January 17  1:50PM ET       W  False  14-2-1    N   
15638  SuperBowl  Sun  January 16  2:50PM ET       L  False  12-4-1    N   
8835   SuperBowl  Sun  January 16  2:50PM ET       W  False    14-3    N   
15656  SuperBowl  Sun  January 14  3:50PM ET       W  False    17-0    N   
...          ...  ...         ...        ...     ...    ...     ...  ...   
28120  SuperBowl  Sun  February 3  6:30PM ET       L  False    15-4    N   
31700  SuperBowl  Sun  February 2  6:30PM ET       L  False    15-4    N   
15509  SuperBowl  Sun  February 2  6:30PM ET       W  False    15-4    N   
15530  SuperBowl  Sun  February 7  6:30PM ET       L  False    16-3    N   
32503  SuperBowl  Sun  February 7  6:30PM ET       W  False    15-5    N   

                        Opp  Score_Tm  ...  Offense_RushY  Offense_TO  \
8817        Baltimore Colts      13.0  ...          104.0         4.0   
6310         Dallas Cowboys      16.0  ...           69.0         7.0   
15638        Dallas Cowboys       3.0  ...           80.0         3.0   
8835         Miami Dolphins      24.0  ...          252.0         1.0   
15656   Washington Redskins      14.0  ...          184.0         2.0   
...                     ...       ...  ...            ...         ...   
28120  New England Patriots       3.0  ...           62.0         1.0   
31700    Kansas City Chiefs      20.0  ...          141.0         2.0   
15509   San Francisco 49ers      31.0  ...          129.0         2.0   
15530  Tampa Bay Buccaneers       9.0  ...          107.0         2.0   
32503    Kansas City Chiefs      31.0  ...          145.0         NaN   

       Defense_1stD  Defense_TotYd  Defense_PassY  Defense_RushY  Defense_TO  \
8817           14.0          329.0          260.0           69.0         7.0   
6310            9.0          217.0          113.0          104.0         4.0   
15638          23.0          352.0          100.0          252.0         1.0   
8835           10.0          185.0          105.0           80.0         3.0   
15656          16.0          228.0           87.0          141.0         3.0   
...             ...            ...            ...            ...         ...   
28120          22.0          407.0          253.0          154.0         1.0   
31700          26.0          397.0          268.0          129.0         2.0   
15509          21.0          351.0          210.0          141.0         2.0   
15530          26.0          340.0          195.0          145.0         NaN   
32503          22.0          350.0          243.0          107.0         2.0   

       Year  Team  OppAbbr  
8817   1970   DAL      CLT  
6310   1970   CLT      DAL  
15638  1971   MIA      DAL  
8835   1971   DAL      MIA  
15656  1972   MIA      WAS  
...     ...   ...      ...  
28120  2018   RAM      NWE  
31700  2019   SFO      KAN  
15509  2019   KAN      SFO  
15530  2020   KAN      TAM  
32503  2020   TAM      KAN  

[102 rows x 24 columns]

It appears that 'N' appears when it is the championship (Super Bowl) game. This makes sense, as it is hosted in a city each year, independently of who is in the game. The easiest solution would be to simply drop the superbowl games, to avoid this problem. Instead, I am going to simply map the 'N' to a '-' for both home and away teams. Because there is only one superbowl game each year, we can still use this to create our `GameID` that will allow us to drop duplicate games (that is more clear in the above DataFrame preview). I will then have to revist the Home and Away team columns after the duplcates have been dropped.

In [32]:
# Function to determine the home team
def is_home(row):
    #is the team (pov) Home? (True, False, N)
    home = row.Home
    
    if home == 'True':
        return row.Team
    elif home == 'False':
        return row.OppAbbr
    elif home == 'N':
        return '-'
    
# Function to determine the away team
def is_away(row):
    #is the team (pov) Home? (True, False, N)
    home = row.Home 
    
    if home == 'False':
        return row.Team
    elif home == 'True':
        return row.OppAbbr
    elif home == 'N':
        return '-'
    


In [33]:
# Create a function to determine who is home and away
df_1970['HomeTeam'] = df_1970.apply(is_home, axis=1) # create home team column
df_1970['AwayTeam'] = df_1970.apply(is_away, axis=1) # create away team column

In [34]:
# Check that it worked as intended
df_1970[['Week','Home','Team','OppAbbr','HomeTeam','AwayTeam']].sample(10)

Week   Home Team OppAbbr HomeTeam AwayTeam
26396             5  False  RAI     NYJ      NYJ      RAI
22819            12   True  OTI     NYG      OTI      NYG
14275            10  False  JAX     DET      DET      JAX
17224             9   True  MIN     GNB      MIN      GNB
9491             13   True  DAL     SEA      DAL      SEA
1861              1  False  BUF     RAV      RAV      BUF
18648            14  False  NWE     CLT      CLT      NWE
25051  Conf. Champ.   True  PIT     OTI      PIT      OTI
8924              7   True  DAL     DET      DAL      DET
7031             17   True  CLT     HTX      CLT      HTX

Everything Looks Good! We can almost create the `GameID` column that joins together `Year`,`Week`,`HomeTeam`,`AwayTeam`. One final task though is to first convert the `Week` to a string, and remap some of the values so they make more sense. (1->01, 'Conf. Champ' -> CC, etc.)

#### Fixing `Week` Column

In [35]:
# Check unique Week values
df_1970.Week.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', 'Wild Card', 'Division', '17', '18',
       'Conf. Champ.', 'SuperBowl'], dtype=object)

In [36]:
# Convert to string 
df_1970['Week'] = df_1970['Week'].astype(str)

In [37]:
# Create mapping dictionary
week_map = {'1': '01', '2':'02','3':'03','4':'04','5':'05','6':'06','7':'07','8':'08','9':'09','Wild Card':'WC','Division':'DI','Conf. Champ.':'CC','SuperBowl':'SB'}

In [38]:
# Map to week column
# df_1970['Week'] = df_1970['Week'].map(week_map)
# df_1970.head()
df_1970['Week'].replace(week_map, inplace=True)

In [39]:
df_1970.Week.unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', 'WC', 'DI', '17', '18', 'CC', 'SB'],
      dtype=object)

#### Create `GameID`
Finally we can join our columns of interest to create a unique game id to be used for dropping duplicate games. 

In [40]:
df_1970['GameID'] = df_1970['Year'].astype(str) + df_1970['Week'] + df_1970['HomeTeam'] + df_1970['AwayTeam']
df_1970.head()

Week  Day          Date       Time Outcome     OT  Rec   Home  \
56   01  Sun  September 20  1:30PM ET       W  False  1-0  False   
57   02  Sun  September 27  2:00PM ET       L  False  1-1  False   
58   03  Sun     October 4  1:00PM ET       W  False  2-1   True   
59   04  Sun    October 11  2:00PM ET       L  False  2-2  False   
60   05  Sun    October 18  4:00PM ET       L  False  2-3  False   

                    Opp  Score_Tm  ...  Defense_TotYd  Defense_PassY  \
56   New Orleans Saints      14.0  ...          235.0          150.0   
57    Green Bay Packers      24.0  ...          344.0          258.0   
58  San Francisco 49ers      21.0  ...          326.0          163.0   
59       Dallas Cowboys       0.0  ...          257.0           39.0   
60       Denver Broncos      10.0  ...          223.0          132.0   

    Defense_RushY  Defense_TO  Year  Team  OppAbbr  HomeTeam  AwayTeam  \
56           85.0         2.0  1970   ATL      NOR       NOR       ATL   
57           86.0         3.0  1970   ATL      GNB       GNB       ATL   
58          163.0         2.0  1970   ATL      SFO       ATL       SFO   
59          218.0         3.0  1970   ATL      DAL       DAL       ATL   
60           91.0         1.0  1970   ATL      DEN       DEN       ATL   

          GameID  
56  197001NORATL  
57  197002GNBATL  
58  197003ATLSFO  
59  197004DALATL  
60  197005DENATL  

[5 rows x 27 columns]

In [41]:
# Check that we have our duplicates
pd.value_counts(df_1970['GameID'])

197001NORATL    2
201003JAXPHI    2
200905SEAJAX    2
200906JAXRAM    2
200908OTIJAX    2
               ..
201713SDGCLE    2
201714CLEGNB    2
201715CLERAV    2
201717PITCLE    2
2020WCWASTAM    2
Name: GameID, Length: 12261, dtype: int64

It seems to have worked just as planned!

In [42]:
# Size check
print(f"Before dropping duplicates the DataFrame has {df_1970.shape[0]} rows.")

Before dropping duplicates the DataFrame has 24522 rows.


#### Drop Duplicate Games

In [43]:
# dropping duplicates
df_no_dup = df_1970.drop_duplicates(subset='GameID').copy()

In [44]:
# Checking new size
print(f"After dropping duplicates the DataFrame has {df_no_dup.shape[0]} rows. This adds up!")

After dropping duplicates the DataFrame has 12261 rows. This adds up!


In [45]:
# Check that it worked for the SuperBowl weeks we were concerned about before
df_no_dup.loc[df_no_dup['Week'] == 'SB'].sort_values(by='Year')

Week  Day        Date       Time Outcome     OT     Rec Home  \
6310    SB  Sun  January 17  1:50PM ET       W  False  14-2-1    N   
8835    SB  Sun  January 16  2:50PM ET       W  False    14-3    N   
15656   SB  Sun  January 14  3:50PM ET       W  False    17-0    N   
15674   SB  Sun  January 13  3:50PM ET       W  False    15-2    N   
16722   SB  Sun  January 12  3:00PM ET       L  False    12-5    N   
8901    SB  Sun  January 18  2:00PM ET       L  False    12-5    N   
16756   SB  Sun   January 9  3:50PM ET       L  False  13-3-1    N   
8935    SB  Sun  January 15  6:00PM ET       W  False    15-2    N   
8955    SB  Sun  January 21  4:00PM ET       L  False    14-5    N   
25052   SB  Sun  January 20  6:00PM ET       W  False    15-4    N   
23715   SB  Sun  January 25  6:00PM ET       L  False    14-5    N   
4202    SB  Sun  January 24  4:00PM ET       L  False    14-5    N   
15816   SB  Sun  January 30  6:00PM ET       L  False    10-3    N   
26198   SB  Sun  January 22  4:50PM ET       W  False    15-4    N   
15854   SB  Sun  January 20  6:00PM ET       L  False    16-3    N   
3349    SB  Sun  January 26  5:00PM ET       W  False    18-1    N   
10133   SB  Sun  January 25  4:00PM ET       L  False    13-6    N   
10152   SB  Sun  January 31  6:00PM ET       L  False  12-5-1    N   
4312    SB  Sun  January 22  5:00PM ET       L  False    14-5    N   
10188   SB  Sun  January 28  5:00PM ET       L  False    13-6    N   
1412    SB  Sun  January 27  5:30PM ET       L  False    15-4    N   
1433    SB  Sun  January 26  6:30PM ET       L  False    15-4    N   
1455    SB  Sun  January 31  6:30PM ET       L  False    14-6    N   
1477    SB  Sun  January 30  6:00PM ET       L  False    14-5    N   
29208   SB  Sun  January 29  6:00PM ET       L  False    13-6    N   
9261    SB  Sun  January 28  6:33PM ET       W  False    15-4    N   
13195   SB  Sun  January 26  6:30PM ET       W  False    16-3    N   
10337   SB  Sun  January 25  6:30PM ET       W  False    16-4    N   
521     SB  Sun  January 31  6:30PM ET       L  False    16-3    N   
22697   SB  Sun  January 30  6:25PM ET       L  False    16-4    N   
20649   SB  Sun  January 28  6:28PM ET       L  False    14-5    N   
19148   SB  Sun  February 3  6:40PM ET       W  False    14-5    N   
26537   SB  Sun  January 26  6:26PM ET       L  False    13-6    N   
2132    SB  Sun  February 1  6:25PM ET       L  False    14-6    N   
19207   SB  Sun  February 6  6:38PM ET       W  False    17-2    N   
25517   SB  Sun  February 5  6:27PM ET       W  False    15-5    N   
3725    SB  Sun  February 4  6:27PM ET       L  False    15-4    N   
19269   SB  Sun  February 3  6:30PM ET       L  False    18-1    N   
8422    SB  Sun  February 1  6:31PM ET       L  False    12-8    N   
6978    SB  Sun  February 7  6:31PM ET       L  False    16-3    N   
13459   SB  Sun  February 6  6:34PM ET       W  False    14-6    N   
19345   SB  Sun  February 5  6:30PM ET       L  False    15-4    N   
28494   SB  Sun  February 3  6:31PM ET       W  False    14-6    N   
10631   SB  Sun  February 2  6:32PM ET       L  False    15-4    N   
19406   SB  Sun  February 1  6:30PM ET       W  False    15-4    N   
2351    SB  Sun  February 7  6:39PM ET       L  False    17-2    N   
846     SB  Sun  February 5  6:30PM ET       L   True    13-6    N   
19468   SB  Sun  February 4  6:30PM ET       L  False    15-4    N   
19489   SB  Sun  February 3  6:30PM ET       W  False    14-5    N   
15509   SB  Sun  February 2  6:30PM ET       W  False    15-4    N   
15530   SB  Sun  February 7  6:30PM ET       L  False    16-3    N   

                        Opp  Score_Tm  ...  Defense_TotYd  Defense_PassY  \
6310         Dallas Cowboys      16.0  ...          217.0          113.0   
8835         Miami Dolphins      24.0  ...          185.0          105.0   
15656   Washington Redskins      14.0  ...          228.0           87.0   
15674     Minnesota Vikings      24.0  ...          238.0    

It worked as intended. I am now going to fill in the `HomeTeam` and `AwayTeam` arbitrarily with the `Team` and `OppAbbr` respectively. This should not affect my analysis, as I will be more concerned with the spread of games, and not necessarily who the home or away team are

In [46]:
# Add the Team value to the HomeTeam column for SB games
df_no_dup.loc[df_no_dup['Week'] == 'SB', 'HomeTeam'] = df_no_dup.loc[df_no_dup['Week'] == 'SB', 'Team']
# Add the OppAbbr value to the AwayTeam column for SB games
df_no_dup.loc[df_no_dup['Week'] == 'SB', 'AwayTeam'] = df_no_dup.loc[df_no_dup['Week'] == 'SB', 'OppAbbr']

# Check results
df_no_dup.loc[df_no_dup['Week'] == 'SB'].sort_values(by='Year')

Week  Day        Date       Time Outcome     OT     Rec Home  \
6310    SB  Sun  January 17  1:50PM ET       W  False  14-2-1    N   
8835    SB  Sun  January 16  2:50PM ET       W  False    14-3    N   
15656   SB  Sun  January 14  3:50PM ET       W  False    17-0    N   
15674   SB  Sun  January 13  3:50PM ET       W  False    15-2    N   
16722   SB  Sun  January 12  3:00PM ET       L  False    12-5    N   
8901    SB  Sun  January 18  2:00PM ET       L  False    12-5    N   
16756   SB  Sun   January 9  3:50PM ET       L  False  13-3-1    N   
8935    SB  Sun  January 15  6:00PM ET       W  False    15-2    N   
8955    SB  Sun  January 21  4:00PM ET       L  False    14-5    N   
25052   SB  Sun  January 20  6:00PM ET       W  False    15-4    N   
23715   SB  Sun  January 25  6:00PM ET       L  False    14-5    N   
4202    SB  Sun  January 24  4:00PM ET       L  False    14-5    N   
15816   SB  Sun  January 30  6:00PM ET       L  False    10-3    N   
26198   SB  Sun  January 22  4:50PM ET       W  False    15-4    N   
15854   SB  Sun  January 20  6:00PM ET       L  False    16-3    N   
3349    SB  Sun  January 26  5:00PM ET       W  False    18-1    N   
10133   SB  Sun  January 25  4:00PM ET       L  False    13-6    N   
10152   SB  Sun  January 31  6:00PM ET       L  False  12-5-1    N   
4312    SB  Sun  January 22  5:00PM ET       L  False    14-5    N   
10188   SB  Sun  January 28  5:00PM ET       L  False    13-6    N   
1412    SB  Sun  January 27  5:30PM ET       L  False    15-4    N   
1433    SB  Sun  January 26  6:30PM ET       L  False    15-4    N   
1455    SB  Sun  January 31  6:30PM ET       L  False    14-6    N   
1477    SB  Sun  January 30  6:00PM ET       L  False    14-5    N   
29208   SB  Sun  January 29  6:00PM ET       L  False    13-6    N   
9261    SB  Sun  January 28  6:33PM ET       W  False    15-4    N   
13195   SB  Sun  January 26  6:30PM ET       W  False    16-3    N   
10337   SB  Sun  January 25  6:30PM ET       W  False    16-4    N   
521     SB  Sun  January 31  6:30PM ET       L  False    16-3    N   
22697   SB  Sun  January 30  6:25PM ET       L  False    16-4    N   
20649   SB  Sun  January 28  6:28PM ET       L  False    14-5    N   
19148   SB  Sun  February 3  6:40PM ET       W  False    14-5    N   
26537   SB  Sun  January 26  6:26PM ET       L  False    13-6    N   
2132    SB  Sun  February 1  6:25PM ET       L  False    14-6    N   
19207   SB  Sun  February 6  6:38PM ET       W  False    17-2    N   
25517   SB  Sun  February 5  6:27PM ET       W  False    15-5    N   
3725    SB  Sun  February 4  6:27PM ET       L  False    15-4    N   
19269   SB  Sun  February 3  6:30PM ET       L  False    18-1    N   
8422    SB  Sun  February 1  6:31PM ET       L  False    12-8    N   
6978    SB  Sun  February 7  6:31PM ET       L  False    16-3    N   
13459   SB  Sun  February 6  6:34PM ET       W  False    14-6    N   
19345   SB  Sun  February 5  6:30PM ET       L  False    15-4    N   
28494   SB  Sun  February 3  6:31PM ET       W  False    14-6    N   
10631   SB  Sun  February 2  6:32PM ET       L  False    15-4    N   
19406   SB  Sun  February 1  6:30PM ET       W  False    15-4    N   
2351    SB  Sun  February 7  6:39PM ET       L  False    17-2    N   
846     SB  Sun  February 5  6:30PM ET       L   True    13-6    N   
19468   SB  Sun  February 4  6:30PM ET       L  False    15-4    N   
19489   SB  Sun  February 3  6:30PM ET       W  False    14-5    N   
15509   SB  Sun  February 2  6:30PM ET       W  False    15-4    N   
15530   SB  Sun  February 7  6:30PM ET       L  False    16-3    N   

                        Opp  Score_Tm  ...  Defense_TotYd  Defense_PassY  \
6310         Dallas Cowboys      16.0  ...          217.0          113.0   
8835         Miami Dolphins      24.0  ...          185.0          105.0   
15656   Washington Redskins      14.0  ...          228.0           87.0   
15674     Minnesota Vikings      24.0  ...          238.0    

In [47]:
# Check proper amount of teams
print(df_no_dup.HomeTeam.nunique())
print(df_no_dup.AwayTeam.nunique())

32
32


## Change 'Offense' and 'Defense' Stats to 'Home' and 'Away' Stats
We still have offense and defense columns (which was based on the team POV). To replace this, we will check if Team == HomeTeam to determine if the stats should be for Home or Away team.

In [48]:
# Quick reminder of the columns we need to change
df_no_dup.columns

Index(['Week', 'Day', 'Date', 'Time', 'Outcome', 'OT', 'Rec', 'Home', 'Opp',
       'Score_Tm', 'Score_Opp', 'Offense_1stD', 'Offense_TotYd',
       'Offense_PassY', 'Offense_RushY', 'Offense_TO', 'Defense_1stD',
       'Defense_TotYd', 'Defense_PassY', 'Defense_RushY', 'Defense_TO', 'Year',
       'Team', 'OppAbbr', 'HomeTeam', 'AwayTeam', 'GameID'],
      dtype='object')

Columns to Change: 
* 'Score_Tm', 
* 'Score_Opp', 
* 'Offense_1stD',
* 'Offense_TotYd',
* 'Offense_PassY', 
* 'Offense_RushY', 
* 'Offense_TO',
* 'Defense_1stD',
* 'Defense_TotYd', 
* 'Defense_PassY', 
* 'Defense_RushY',
* 'Defense_TO'

In [49]:
# Create columns for home and away scores
df_no_dup['HomeScore'] = df_no_dup.apply(lambda x: x.Score_Tm if x.Team == x.HomeTeam else x.Score_Opp, axis=1)
df_no_dup['AwayScore'] = df_no_dup.apply(lambda x: x.Score_Tm if x.Team == x.AwayTeam else x.Score_Opp, axis=1)

In [50]:
df.columns

Index(['Week', 'Day', 'Date', 'Time', 'Outcome', 'OT', 'Rec', 'Home', 'Opp',
       'Score_Tm', 'Score_Opp', 'Offense_1stD', 'Offense_TotYd',
       'Offense_PassY', 'Offense_RushY', 'Offense_TO', 'Defense_1stD',
       'Defense_TotYd', 'Defense_PassY', 'Defense_RushY', 'Defense_TO', 'Year',
       'Team'],
      dtype='object')

In [51]:
# Check that it worked
df_no_dup[['Team','OppAbbr','Home','Score_Tm','Score_Opp','HomeTeam','AwayTeam','HomeScore','AwayScore']].sample(5)

Team OppAbbr   Home  Score_Tm  Score_Opp HomeTeam AwayTeam  HomeScore  \
6954   CLT     JAX  False      31.0       24.0      JAX      CLT       24.0   
10214  DEN     NWE  False       9.0        6.0      NWE      DEN        6.0   
15766  MIA     NYJ   True      24.0       27.0      MIA      NYJ       24.0   
13070  GNB     SFO   True      20.0       24.0      GNB      SFO       20.0   
13993  HTX     KAN  False      20.0       34.0      KAN      HTX       34.0   

       AwayScore  
6954        31.0  
10214        9.0  
15766       27.0  
13070       24.0  
13993       20.0

In [52]:
# Same process for the rest of columns
# Start with matching Offensive Stats with Home Stats
df_no_dup['Home1D'] = df_no_dup.apply(lambda x: x.Offense_1stD if x.Team == x.HomeTeam else x.Defense_1stD, axis=1) # Home Teams 1st Downs
df_no_dup['HomeTotYd'] = df_no_dup.apply(lambda x: x.Offense_TotYd if x.Team == x.HomeTeam else x.Defense_TotYd, axis=1) # Home Teams Total Yards
df_no_dup['HomePassYd'] = df_no_dup.apply(lambda x: x.Offense_PassY if x.Team == x.HomeTeam else x.Defense_PassY, axis=1) # Home Teams Pass Yards
df_no_dup['HomeRushYd'] = df_no_dup.apply(lambda x: x.Offense_RushY if x.Team == x.HomeTeam else x.Defense_RushY, axis=1) # Home Teams Rush Yards
df_no_dup['HomeTO'] = df_no_dup.apply(lambda x: x.Offense_TO if x.Team == x.HomeTeam else x.Defense_TO, axis=1) # Home Teams Time Outs
# Now for Away Team
df_no_dup['Away1D'] = df_no_dup.apply(lambda x: x.Offense_1stD if x.Team == x.AwayTeam else x.Defense_1stD, axis=1) # Away Teams 1st Downs
df_no_dup['AwayTotYd'] = df_no_dup.apply(lambda x: x.Offense_TotYd if x.Team == x.AwayTeam else x.Defense_TotYd, axis=1) # Away Teams Total Yards
df_no_dup['AwayPassYd'] = df_no_dup.apply(lambda x: x.Offense_PassY if x.Team == x.AwayTeam else x.Defense_PassY, axis=1) # Away Teams Pass Yards
df_no_dup['AwayRushYd'] = df_no_dup.apply(lambda x: x.Offense_RushY if x.Team == x.AwayTeam else x.Defense_RushY, axis=1) # Away Teams Rush Yards
df_no_dup['AwayTO'] = df_no_dup.apply(lambda x: x.Offense_TO if x.Team == x.AwayTeam else x.Defense_TO, axis=1) # Away Teams Time Outs

In [53]:
# Check that everything makes sense
pd.set_option('display.max_columns', None)
df_no_dup.head()

Week  Day          Date       Time Outcome     OT  Rec   Home  \
56   01  Sun  September 20  1:30PM ET       W  False  1-0  False   
57   02  Sun  September 27  2:00PM ET       L  False  1-1  False   
58   03  Sun     October 4  1:00PM ET       W  False  2-1   True   
59   04  Sun    October 11  2:00PM ET       L  False  2-2  False   
60   05  Sun    October 18  4:00PM ET       L  False  2-3  False   

                    Opp  Score_Tm  Score_Opp  Offense_1stD  Offense_TotYd  \
56   New Orleans Saints      14.0        3.0          14.0          265.0   
57    Green Bay Packers      24.0       27.0          19.0          384.0   
58  San Francisco 49ers      21.0       20.0          18.0          356.0   
59       Dallas Cowboys       0.0       13.0           9.0          128.0   
60       Denver Broncos      10.0       24.0          15.0          216.0   

    Offense_PassY  Offense_RushY  Offense_TO  Defense_1stD  Defense_TotYd  \
56          116.0          149.0         2.0           9.0          235.0   
57          272.0          112.0         5.0          15.0          344.0   
58          204.0          152.0         2.0          18.0          326.0   
59           44.0           84.0         3.0          13.0          257.0   
60          140.0           76.0         5.0          18.0          223.0   

    Defense_PassY  Defense_RushY  Defense_TO  Year Team OppAbbr HomeTeam  \
56          150.0           85.0         2.0  1970  ATL     NOR      NOR   
57          258.0           86.0         3.0  1970  ATL     GNB      GNB   
58          163.0          163.0         2.0  1970  ATL     SFO      ATL   
59           39.0          218.0         3.0  1970  ATL     DAL      DAL   
60          132.0           91.0         1.0  1970  ATL     DEN      DEN   

   AwayTeam        GameID  HomeScore  AwayScore  Home1D  HomeTotYd  \
56      ATL  197001NORATL        3.0       14.0     9.0      235.0   
57      ATL  197002GNBATL       27.0       24.0    15.0      344.0   
58      SFO  197003ATLSFO       21.0       20.0    18.0      356.0   
59      ATL  197004DALATL       13.0        0.0    13.0      257.0   
60      ATL  197005DENATL       24.0       10.0    18.0      223.0   

    HomePassYd  HomeRushYd  HomeTO  Away1D  AwayTotYd  AwayPassYd  AwayRushYd  \
56       150.0        85.0     2.0    14.0      265.0       116.0       149.0   
57       258.0        86.0     3.0    19.0      384.0       272.0       112.0   
58       204.0       152.0     2.0    18.0      326.0       163.0       163.0   
59        39.0       218.0     3.0     9.0      128.0        44.0        84.0   
60       132.0        91.0     1.0    15.0      216.0       140.0        76.0   

    AwayTO  
56     2.0  
57     5.0  
58     2.0  
59     3.0  
60     5.0

Everything looks great - matching up just as expected! All we need to do left to have a clean DataFrame for my analysis is to drop the unwanted / Redundant Columns

## Drop Unwanted Columns, and export clean DataFrame

In [54]:
# Only take the columns we want for our analysis (ignoring redunant)
cols = ['GameID','Year','Week','Day','HomeTeam','HomeScore','AwayScore','AwayTeam','Home1D','HomeTotYd','HomePassYd','HomeRushYd','HomeTO','Away1D','AwayTotYd','AwayPassYd','AwayRushYd','AwayTO']
df_clean = df_no_dup.loc[:,cols].sort_values(by=['Year','Week']).reset_index(drop=True)
df_clean.head()

GameID  Year Week  Day HomeTeam  HomeScore  AwayScore AwayTeam  \
0  197001NORATL  1970   01  Sun      NOR        3.0       14.0      ATL   
1  197001BUFDEN  1970   01  Sun      BUF       10.0       25.0      DEN   
2  197001NYGCHI  1970   01  Sat      NYG       16.0       24.0      CHI   
3  197001CINRAI  1970   01  Sun      CIN       31.0       21.0      RAI   
4  197001CLENYJ  1970   01  Mon      CLE       31.0       21.0      NYJ   

   Home1D  HomeTotYd  HomePassYd  HomeRushYd  HomeTO  Away1D  AwayTotYd  \
0     9.0      235.0       150.0        85.0     2.0    14.0      265.0   
1     8.0      149.0        56.0        93.0     4.0    11.0      282.0   
2    16.0      311.0       253.0        58.0     3.0    18.0      256.0   
3    14.0      364.0       117.0       247.0     1.0    13.0      291.0   
4    20.0      221.0       145.0        76.0     NaN    31.0      454.0   

   AwayPassYd  AwayRushYd  AwayTO  
0       116.0       149.0     2.0  
1       146.0       136.0     3.0  
2       158.0        98.0     3.0  
3       243.0        48.0     2.0  
4       286.0       168.0     4.0

In [55]:
df_clean.shape

(12261, 18)

In [56]:
# Export to Clean CSV
df_clean.to_csv('data/nfl_scores_clean.csv', index=False)

In [57]:
df_clean['Week'].value_counts()

01    751
02    751
13    751
14    750
12    746
11    730
03    702
10    695
04    688
05    680
09    678
06    677
07    676
08    672
15    647
16    647
17    493
DI    204
WC    156
CC    102
SB     51
18     14
Name: Week, dtype: int64